# Scraping IGN

In [110]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)

def loadAndScroll(browser, url, scrolls, t):
    browser.get(url)

    for i in range(scrolls):
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(t)
    
    doc = BeautifulSoup(browser.page_source, 'html.parser')
    return doc
        

In [29]:
url = "https://nordic.ign.com/article/review?keyword__type=game"
doc = loadAndScroll(browser, url, 20,1)
headlines = doc.find_all('h3')

In [30]:

with open('linksIGN.txt','w') as f:
    for headline in headlines:
        link = headline.find('a')
        if link:
            f.write(str(link.get('href'))+ '\n')
            #print(link.get('href'))
            

In [ ]:
import re
import csv

pattern = re.compile(r'<p id="id_keyword__.*?__text">')
rows = [['Game', 'Score', 'Text']]
with open('linksIGN.txt','r') as f:
    i = 0
    for url in f:
        doc = loadAndScroll(browser,url, 0, 0.0)
        score = doc.find_all('span', class_="side-wrapper side-wrapper hexagon-content")
        try:
            #ugly hack but works
            score = str(score).split('div')[1]
            if score[2] == '<':
                score = score[1]
            else:
                score = score[1:3] #10
            
        except:
            continue
            score = "-1"
        title = str(doc.find_all('title')[0])
        title = title.strip('<title>').strip('Review</title>')
        
        text = doc.find_all('p')
        review_text = ''
        for section in text:
            t = str(section)
            review_text += t + ' '
        
        rows.append([title, score, review_text])
        
        i += 1
        print(i)
        
with open('IgnReviews.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerows(rows)

In [85]:
with open('IgnReviews2.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerows(rows)

In [94]:
import pandas as pd
df = pd.read_csv('IgnReviews.csv')
pattern = re.compile(r'<(.*?)>')

for entry in df.itertuples():
    txt = entry.Text
    new = txt
    matches =  re.finditer(pattern, str(txt))
    for m in matches:
        try:
            new = re.sub(m.group(),'',new)
        except:
            print(entry.Index, entry.Game, m)
    
    df.loc[entry.Index,'Text'] = new

df.to_csv('IgnClean2.csv')
    

# Scrape Gamestop

In [ ]:
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)

baseUrl = "https://www.gamespot.com/games/reviews/?page="
with open('linksGamespot.txt','w') as f:
    for i in range(1,201):
        url = baseUrl + str(i)
        doc = loadAndScroll(browser, url, 1,0.5)
        links = doc.find_all('a')
        for link in links:
            class_ = link.get('class')
            if class_ == ['card-item__link', 'text-decoration--none']:
                href = link.get('href')
                f.write("https://www.gamespot.com" + href + '\n')

In [107]:
import re

def getScoreGS(divs):
    score = "-1"
    for d in divs:
        class_ = d.get('class')
        if class_ == ['review-ring-score__score', 'text-bold']:
            score = str(d).strip("</div>")
            idx = score.find('>') + 1
            score = score[idx:]
    return score
            
def getTextGS_og(texts):
    txt = ""
    for p in texts:
        print(p)
        if p.get('dir') == "ltr":
            t = str(p)
            
            t = t.strip('<p dir=\"ltr\">').strip("</p>")
            txt += t
    return txt

def getTextGS(divs):
    txt = ""
    for d in divs:
        class_ = d.get('class')
        if class_ == ['js-content-entity-body']:
            ps = d.find_all('p')
            ps = [str(x) for x in ps]
            indices = []
            for i,p in enumerate(ps):
                if "By clicking 'enter', you agree to GameSpot's" in p:
                    indices.append(i)
                if "Want us to remember this setting for all your devices?" in p:
                    indices.append(i)
                if "class=\"video-caption\"" in p:
                    indices.append(i)
                if "button class=\"js-vid-play" in p:
                    indices.append(i)
                if "class=\"av-video-player-no-js" in p:
                    indices.append(i)
                if "Sign in</a> now!" in p:
                    indices.append(i)
                
            strs = [v for i, v in enumerate(ps) if i not in indices]
                    
            txt = ' '.join(strs)
            #print(txt)
       
    return txt


def getGameGS(lists):
    pattern = re.compile(r'<span>(.*?)</span>')
    game = ""
    for li in lists:
        #print(li.get('class') )
        if li.get('class') == ['subnav-list__item', 'subnav-list__item-primary']:
            string = str(li)
            match = re.search(pattern, string)
            if match:
                game = match.group(1).strip()
    return game


In [84]:
#Clear all double entries

f =  open('linksGamespot.txt','r')
rows = f.readlines()

last = []
with open('linksGSClean.txt','w') as f:
    for link in rows:
        link_str = str(link)
        current = link_str.split('/')
        current = current[:-2]
        
        if last == current:
            #print(current[-1], last[-1])
            continue
        last = current
        f.write(link)

In [113]:
import csv

"""row = ['Game', 'Score', 'Text']
with open('GamespotReviewTest.csv', 'a') as csvfile:
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerow(row)"""
        

with open('linksGSClean.txt','r') as f:
    urls =  f.readlines()
tot = len(urls)
with open('GamespotReviewTest.csv', 'a') as csvfile:
    csvwriter = csv.writer(csvfile)  
    for i, url in enumerate(urls):
        #temp fix
        if i < 2910:
            continue
        
        doc = loadAndScroll(browser,url, 1, 2)
        divs = doc.find_all('div')
        texts = doc.find_all('p')
        lists = doc.find_all('li')
        game = getGameGS(lists)
        score = getScoreGS(divs)
        text = getTextGS(divs)
        row = [game, score, text]
        csvwriter.writerow(row)
            
        print(str(i) + '/' + str(tot))



2910/3840
2911/3840
2912/3840
2913/3840
2914/3840
2915/3840
2916/3840
2917/3840
2918/3840
2919/3840
2920/3840
2921/3840
2922/3840
2923/3840
2924/3840
2925/3840
2926/3840
2927/3840
2928/3840
2929/3840
2930/3840
2931/3840
2932/3840
2933/3840
2934/3840
2935/3840
2936/3840
2937/3840
2938/3840
2939/3840
2940/3840
2941/3840
2942/3840
2943/3840
2944/3840
2945/3840
2946/3840
2947/3840
2948/3840
2949/3840
2950/3840
2951/3840
2952/3840
2953/3840
2954/3840
2955/3840
2956/3840
2957/3840
2958/3840
2959/3840
2960/3840
2961/3840
2962/3840
2963/3840
2964/3840
2965/3840
2966/3840
2967/3840
2968/3840
2969/3840
2970/3840
2971/3840
2972/3840
2973/3840
2974/3840
2975/3840
2976/3840
2977/3840
2978/3840
2979/3840
2980/3840
2981/3840
2982/3840
2983/3840
2984/3840
2985/3840
2986/3840
2987/3840
2988/3840
2989/3840
2990/3840
2991/3840
2992/3840
2993/3840
2994/3840
2995/3840
2996/3840
2997/3840
2998/3840
2999/3840
3000/3840
3001/3840
3002/3840
3003/3840
3004/3840
3005/3840
3006/3840
3007/3840
3008/3840
3009/3840


In [122]:
import pandas as pd
df = pd.read_csv('GSClean2.csv')
pattern = re.compile(r'a href=(.*)>')

for entry in df.itertuples():
    txt = entry.Text
    new = txt
    matches =  re.finditer(pattern, str(txt))
    for m in matches:
        try:
            new = re.sub(m.group(),'',new)
        except:
            print(entry.Index, entry.Game, m)
        print(entry.Index, entry.Game, m)
    
    df.loc[entry.Index,'Text'] = new
header = ['Game','Score','Text']
df.to_csv('GSClean2.csv', columns=header)

402 Hitman 3 <re.Match object; span=(5148, 5285), match='a href="https://www.gamespot.com/articles/hitman->
422 Ori and the Will of the Wisps <re.Match object; span=(982, 1096), match='a href="https://www.gamespot.com/articles/what-do>
493 Grounded <re.Match object; span=(247, 299), match='a href="https://www.gamespot.com/games/subnautica>
529 XCOM 2 <re.Match object; span=(666, 714), match='a href="https://www.gamespot.com/games/xcom-2/">'>
541 WHAT THE GOLF? <re.Match object; span=(0, 75), match='a href="http://whatthegolf.com/" rel="nofollow no>
587 The Outer Worlds <re.Match object; span=(0, 52), match='a href="https://www.gamespot.com/the-outer-worlds>
612 Super Monkey Ball: Banana Blitz <re.Match object; span=(0, 66), match='a href="https://www.gamespot.com/super-monkey-bal>
645 Monster Hunter: World <re.Match object; span=(0, 56), match='a href="https://www.gamespot.com/monster-hunter-w>
649 The Dark Pictures Anthology: Man of Medan <re.Match object; span=(0, 66), match='a href=

# PC GAMER

In [113]:
urlArchive = "https://www.pcgamer.com/reviews/archive/"

options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)



In [114]:
doc = loadAndScroll(browser, urlArchive, 1,0.5)
li = doc.find_all('ul')
with open('linksPCGarchive.txt','w') as f:
    for l in li:
        as_ = l.find_all('a')
        for a in as_:
            link = a['href']
            link_str = str(link)
            if link_str.startswith("https://www.pcgamer.com/reviews/archive/"):
                f.write(link_str + '\n')

In [ ]:
f =  open('linksPCGarchive.txt','r')
rows = f.readlines()

with open('linksPCG.txt','w') as f:
    for link in rows:
        doc = loadAndScroll(browser, link, 1,0.5)
        lists = doc.find_all('li')
        for li in lists:
            class_ = li.get('class')
            if class_ == ['day-article']:
                a = li.find_all('a')
                f.write(a[0]['href'] + '\n')


Remove entries containing specific word

In [219]:
f =  open('linksPCGClean.txt','r')
rows = f.readlines()

with open('linksPCGClean.txt','w') as f:
    for link in rows:
        if "gainward" in link:
            print(link)
            continue
        f.write(link)

In [8]:
test = "https://www.gamespot.com/reviews/call-of-duty-4-modern-warfare-review/1900-6182426/"

doc =loadAndScroll(browser,test,1,1)
texts = doc.find_all('p')
text = getTextGS(texts)
print(text)